In [2]:
!nvidia-smi

Mon Oct 10 12:31:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          Off  | 00000000:89:00.0 Off |                    0 |
|  0%   36C    P0    71W / 300W |      0MiB / 46068MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import random
from importlib import reload

import torch

from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForCausalLM

from util import mt

In [5]:
model_name = 'EleutherAI/gpt-neo-125M'
model_name = 'facebook/xglm-4.5B'
model_name = 'facebook/xglm-2.9B'
model_name = 'facebook/xglm-7.5B'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).eval().half().cuda()

In [55]:
src = load_dataset("gsarti/flores_101", 'zho_simpl')
tgt = load_dataset("gsarti/flores_101", 'eng')
eng = load_dataset("gsarti/flores_101", 'eng')

Found cached dataset flores_101 (/gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/zho_simpl/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335)


  0%|          | 0/2 [00:00<?, ?it/s]

Found cached dataset flores_101 (/gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/eng/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335)


  0%|          | 0/2 [00:00<?, ?it/s]

Found cached dataset flores_101 (/gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/eng/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335)


  0%|          | 0/2 [00:00<?, ?it/s]

In [56]:
src_lang = '中文'
tgt_lang = 'English'

In [9]:
def tokenize(s):
  ids = []
  i = 0
  t = 0
  while t < len(s):
    c = s[t]
    if c == '\n':
      ids += s_ids(s[i:t]) + [2]
      i = t+1
      t = t+1
    else:
      t += 1
  ids += s_ids(s[i:])
  return ids

In [10]:
def s_ids(s):
  return tokenizer(s).input_ids[1:]

In [11]:
def make_demo_prompt(src_endonym, tgt_endonym, src, tgt, k):
  src_points = random.sample(list(src), k=k)
  tgt_points = [ tgt[point['id']-1] for point in src_points ]
  demos = []
  for s, t in zip(src_points, tgt_points):
    demo = f"{src_endonym} : {s['sentence']} {tgt_endonym} : {t['sentence']}"
    demos.append(demo)
  demo_prompt_ids = [2]
  for demo in demos:
    demo_prompt_ids.extend(s_ids(demo))
    demo_prompt_ids.append(2)
  return demo_prompt_ids

In [29]:
reload(mt)

<module 'util.mt' from '/mmfs1/gscratch/zlab/ahai/repo/infogain/util/mt.py'>

In [76]:
src_points = random.sample(list(src['dev']), k=10)
tgt_points = [ tgt['dev'][point['id']-1] for point in src_points ]

In [77]:
demo_prompt_ids = mt.make_demo_prompt(src_points, tgt_points, src_lang, tgt_lang, tokenizer)
uncond_demo_prompt_ids = mt.make_uncond_demo_prompt(tgt_points, tokenizer)

In [81]:
idx += 1
inference_point = src['dev'][idx] 
cond_prompt_ids = demo_prompt_ids + s_ids(f"{src_lang} : {inference_point['sentence']} {tgt_lang} :")
cond_prompt_t = torch.LongTensor([cond_prompt_ids]).to(model.device)
uncond_prompt_ids = demo_prompt_ids + s_ids(f"{tgt_lang} :")
uncond_prompt_t = torch.LongTensor([uncond_prompt_ids]).to(model.device)
print(src['dev'][idx]['sentence'])
print()
print(tgt['dev'][idx]['sentence'])
print()
print(eng['dev'][idx]['sentence'])

伊尔-76 自 20 世纪 70 年代以来，一直是俄罗斯和苏联军队的主要组成部分，上个月在俄罗斯已经发生过一起严重事故。

The Il-76 has been a major component of both the Russian and Soviet military since the 1970s, and had already seen a serious accident in Russia last month.

The Il-76 has been a major component of both the Russian and Soviet military since the 1970s, and had already seen a serious accident in Russia last month.


In [82]:
with torch.no_grad():
  gen_ids = mt.cfg_k_greedy(model, 0.3, cond_prompt_t, uncond_prompt_t, 50, k=3)
print(tokenizer.decode(gen_ids[0].tolist()))

The Il-76 has been in service since the 1970s, and in Russia last month there was a serious accident.</s> 中文 : 他们在那里呆了两个星期,然后他们就离开了。 English : They stayed there for two


In [83]:
with torch.no_grad():
  greedy_gen_ids = mt.greedy(model, cond_prompt_t, 50)
print(tokenizer.decode(greedy_gen_ids[0].tolist()))

The Il-76 has been in service since the 1970s, and it has been involved in several serious accidents in Russia.</s> 中文 : 他们在那里呆了两个星期,然后他们就离开了。 English : They stayed there for


In [75]:
tokenizer.decode(cond_prompt_t[0])

'</s> 中文 : 퓨어랜드 종이접기는 한 번에 접기는 한 번만 해야 하고, 더 복잡한 뒤집어 접기는 허용되지 않으며 모든 접기는 단순한 위치여야 한다는 제한이 있는 종이접기 방식이다. English : Pureland-origamiin liittyy sellaisia rajoituksia, että kerrallaan voidaan tehdä vain yksi taitos, monimutkaisemmat taitokset, kuten käänteiset taitokset, ovat kiellettyjä, ja kaikilla taitoksilla on suoraviivainen paikka.</s> 中文 : "전 세계 대부분에선 손을 흔드는 것은 ""안녕하세요""를 나타내는 친절한 제스처입니다." English : Monissa osissa maailmaa vilkuttaminen on ystävällinen ele, joka merkitsee tervehtimistä.</s> 中文 : 이 분야의 애플리케이션 중 일부는 이용자가 스마트폰을 대상을 향하게 하면, 표지판 등에 적힌 외국어 텍스트를 번역하기도 한다. English : Jotkin tähän luokkaan kuuluvat sovellukset pystyvät jopa kääntämään vieraskielistä tekstiä kylteistä tai muista reaalimaailman kohteista, kun käyttäjä osoittaa niihin älypuhelimellaan.</s> 中文 : 사막의 한더위도 밤에는 <unk>시 매서운 추위로 변할 수 있다. 따뜻한 옷 없이 저체온증은 실질적인 위험이다. English : Jopa kuumimmat autiomaat voivat olla äärimmäisen kylmiä yöllä. Hypotermia on todellinen riski ilman lämpimiä vaatteita.</s> 中文 : 사유지나 여느 도시에 텐트를 설치하

In [778]:
gen_ids

tensor([[   218,   2686,    113,     88,    256,  14170,    753,     48,     32,
           1075,    316,     20,     67,     11,  19282,  16740,     22,     32,
         122605,      5,      2,  73913,    129,    218,     25, 168217,    395,
            824,    102,  54862,    522,    247,  11076,  66092, 141249,  40165,
             64,   2494,    476,    218,     25,  46298,   5281,     27,    102,
           2774,      5,   7614,    129,    218]], device='cuda:0')

In [775]:
prompt_ids = torch.LongTensor([tokenizer('translate into english, 中文： 谢谢你！').input_ids[:-1]]).to(model.device)

In [232]:
prompt_ids = torch.LongTensor([tokenize('中文： 谢谢你! english:')]).to(model.device)                          

In [233]:
prompt_ids.tolist()

[[6, 19773, 13, 6, 13305, 13305, 710, 35, 83763, 13]]

In [351]:
model.config

XGLMConfig {
  "_name_or_path": "facebook/xglm-2.9B",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "XGLMForCausalLM"
  ],
  "attention_dropout": 0.1,
  "attention_heads": 16,
  "bos_token_id": 0,
  "d_model": 2048,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "eos_token_id": 2,
  "ffn_dim": 8192,
  "init_std": 0.02,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "xglm",
  "num_layers": 48,
  "pad_token_id": 1,
  "scale_embedding": true,
  "transformers_version": "4.22.2",
  "use_cache": true,
  "vocab_size": 256008
}

In [275]:
with torch.no_grad():
  gen_ids = mt.sample(model, prompt_ids, 200)

TypeError: expected Tensor as element 0 in argument 0, but got list

In [309]:
tokenizer.decode(gen_ids[0].tolist())

'Leading researchers say that the test can detect cancer, tuberculosis, HIV and malaria in patients who live in low-income countries, where for example, the survival rate of breast cancer is half that of the richer countries.</s> Finnish.</s> Finnish : Hän sanoi, että hän sanoi, että hän sanoi, että, että, että, että, että, että, että, että, että, että, että, että, että, että, että, että,'

In [231]:
print(tokenizer.decode(prompt_ids[0]))

中文: 谢谢你! | english:


In [14]:
inference_point['sentence']

'Stanfordin yliopiston lääketieteen laitoksen tutkijat ilmoittivat maanantaina uuden diagnostiikkatyökalun keksimisestä: solut tyypin mukaan lajitteleva pienenpieni tulostettava siru, joka voidaan valmistaa normaaleilla mustesuihkutulostimilla mahdollisesti noin yhden Yhdysvaltain sentin kappalehintaan.'

In [209]:
s_ids('')

[]

In [219]:
def tokenize(s):
  ids = []
  i = 0
  t = 0
  while t < len(s):
    c = s[t]
    if c == '\n':
      ids += s_ids(s[i:t]) + [2]
      i = t+1
      t = t+1
    else:
      t += 1
  ids += s_ids(s[i:])
  return ids

In [12]:
vocab = tokenizer.get_vocab()
tokens = list(sorted(tokenizer.get_vocab(), key=lambda k: vocab[k]))

In [121]:
tokens[:15]

['<s>',
 '<pad>',
 '</s>',
 '<unk>',
 ',',
 '.',
 '▁',
 's',
 '-',
 'a',
 '▁de',
 '▁a',
 'e',
 ':',
 'i']

In [305]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_bos_token',
 '_call_one',
 '_cls_token',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager',
 '_mask_token',
 '_pad',
 '_pad_token',
 '_pad

In [306]:
tokenizer.model_max_length

1000000000000000019884624838656

In [319]:
def make_demo_prompt(src_points, tgt_points, src_name, tgt_name, src, tgt, k, tokenizer):
    demos = []
    for s, t in zip(src_points, tgt_points):
        assert(s['id'] == t['id'])
        demo = f"{src_name} : {s['sentence']} {tgt_name} : {t['sentence']}"
        demos.append(demo)
    demo_prompt_ids = [2]
    for demo in demos:
      demo_prompt_ids.extend(mt.xglm_tokenize(demo, tokenizer))
      demo_prompt_ids.append(2)
    return demo_prompt_ids

In [323]:
src_points = random.sample(list(src['dev']), k=3)
tgt_points = [ tgt['dev'][point['id']-1] for point in src_points ]

In [334]:
reload(mt)

<module 'util.mt' from '/mmfs1/gscratch/zlab/ahai/repo/infogain/util/mt.py'>

In [330]:
print(mt.post_proc(tokenizer.decode(mt.make_demo_prompt(src_points, tgt_points, 'finnish', 'english', tokenizer))))


 finnish : Kiertoajelut ovat halvempia suurille ryhmille, joten jos liikut yksin tai vain yhden ystävän kanssa, koeta tutustua muihin ihmisiin ja muodostaa 4–6 hengen ryhmä parempaa yksilöhintaa varten. english : Tours are cheaper for larger groups, so if you're by yourself or with just one friend, try to meet other people and form a group of four to six for a better per-person rate.
 finnish : Vaatimukset on tarkoitettu tekemään maiden välisestä muuttovirrasta järjestelmällisen. english : These requirements are designed to provide an organized migratory flow between both countries.
 finnish : Varo: pikkukaupunkien baarit eivät täälläpäin aina ole parhaita ajanviettopaikkoja vieraspaikkakuntalaisille. english : Beware: small-town bars here are not always good places for the out-of-state visitor to hang out.



In [331]:
def make_uncond_demo_prompt(tgt_points, tokenizer):
    uncond_demo_prompt_ids = [2]
    for t in tgt_points:
      uncond_demo_prompt_ids.extend(mt.xglm_tokenize(t['sentence'], tokenizer))
      uncond_demo_prompt_ids.append(2)
    return uncond_demo_prompt_ids

In [333]:
print(mt.post_proc(tokenizer.decode(make_uncond_demo_prompt(tgt_points, tokenizer))))


 Tours are cheaper for larger groups, so if you're by yourself or with just one friend, try to meet other people and form a group of four to six for a better per-person rate.
 These requirements are designed to provide an organized migratory flow between both countries.
 Beware: small-town bars here are not always good places for the out-of-state visitor to hang out.

